## Word Vectorisation Notebook
Below is the code used for generating various embeddings used in the neural models.

### Imported libraries

In [5]:
import os
import urllib.request
import numpy as np
import zipfile
import tensorflow as tf
import collections
import random
import h5py as h5py


from keras.models import Sequential,Model
from keras.optimizers import RMSprop
from keras.layers import Embedding,LSTM,Dense,Lambda,merge,Input
from keras.callbacks import TensorBoard,ModelCheckpoint,Callback
from keras import backend as K

### Functions defined
```python
maybe_download(filename)
#Downloads a file if not present.

read_data(filename)
#Extract the first file enclosed in a zip file as a list of words.

build_dataset(words, n_words)
#Process raw inputs into a dataset.

generate_batches(data, size, contextWidth, negativeSize)
#Returns batches of input words with their contexts and a set of negative samples.
```


In [6]:
def maybe_download(url, filename):
  """Download a file if not present."""
  if not os.path.exists("./downloads/"+filename):
    filename, _ = urllib.request.urlretrieve(url + filename, "./downloads/"+filename)
  return filename

def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words."""
  with zipfile.ZipFile("./downloads/"+filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

def generate_batches(data, size, contextWidth, negativeSize):
  cHalfWidth = int(contextWidth/2)
  words = []
  contexts = []
  negatives = []
  index = random.sample(range(cHalfWidth,len(data)-cHalfWidth),size)
  for z in index:
      context = []
      for m in range(-cHalfWidth,cHalfWidth+1):
        if m == 0:
          words.append([data[z]])
        else: 
          context.append(data[z+m])
      contexts.append(context)
      negatives.append(random.sample(data,negativeSize))
  return([np.array(words),np.array(contexts),np.array(negatives)],[np.array([1]*size),np.array([[0]*negativeSize]*size)])

### Downloading Wikipedia text database

In [7]:
filename = maybe_download('http://mattmahoney.net/dc/', 'text8.zip')
vocabulary = read_data(filename)
print('Number of words: ', len(vocabulary))

Number of words:  17005207


### Parameters for skipgram model

In [8]:
vocabulary_size = 5000
data_index = 0
batch_size = 128
wordvec_dim = 32

skip_window = 3       # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.
context_half = 3
context_size = context_half*2

valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
neg_size = 5    # Number of negative examples to sample.


### Creating a dictionary and reverse dictionary for word embedding

In [9]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary,vocabulary_size)
del vocabulary  # Hint to reduce memory.

### Saving a .tsv label file for viewing in TensorBoard

In [10]:
with open('./logs/word2vec_label.tsv', 'w') as fr:
    for i in range(vocabulary_size):
        fr.write(reverse_dictionary[i]+'\n')

### Generate training/validation batches

In [11]:
X,Y = generate_batches(data, 500000, context_size, neg_size)
vX, vY = generate_batches(data, 5000, context_size, neg_size)

### Definining neural model with Keras
Graph of word2vec neural model used shown below.


![Graph of cbow model](./images/word2vecmodel.png)


In [12]:
word = Input(shape=(1,), name='inputWord')
context = Input(shape=(context_size,), name='inputContext')
negSamples = Input(shape=(neg_size,), name='inputNegatives')

word2vec = Embedding(input_dim=vocabulary_size,output_dim=wordvec_dim, embeddings_initializer='glorot_normal', name='word2vec')

vec_word = word2vec(word)
vec_context = word2vec(context)
vec_negSamples = word2vec(negSamples)
cbow = Lambda(lambda x: K.mean(x, axis=1), name='cbowAverage')(vec_context)

word_context = merge([vec_word, cbow], mode='dot')
negative_context = merge([vec_negSamples, cbow], mode='dot', concat_axis=-1)

model = Model(input=[word,context,negSamples], output=[word_context,negative_context])

/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/keras/legacy/layers.py:458: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  del sys.path[0]
/Applications/anaconda/envs/nlp/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  from i

In [13]:
model.compile(optimizer='rmsprop', loss='mse', metrics=['accuracy'])
model.summary()
from keras.utils import plot_model
plot_model(model, to_file='./images/word2vecmodel.png', show_shapes=True)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputWord (InputLayer)          (None, 1)            0                                            
__________________________________________________________________________________________________
inputContext (InputLayer)       (None, 6)            0                                            
__________________________________________________________________________________________________
word2vec (Embedding)            multiple             160000      inputWord[0][0]                  
                                                                 inputContext[0][0]               
                                                                 inputNegatives[0][0]             
__________________________________________________________________________________________________
inputNegat

### Create logs for saving parameters and run training

In [14]:
tensorboard = TensorBoard(log_dir='./logs/wordvec', 
batch_size=500, histogram_freq=1, write_images=True, write_grads=False, write_graph=True, embeddings_freq=1)

model_checkpoint = ModelCheckpoint('./logs/wordvec_model.h5')

model.fit(X,Y,epochs=50,batch_size=500,callbacks=[model_checkpoint,tensorboard], validation_data=(vX,vY))

Train on 500000 samples, validate on 5000 samples
Epoch 1/50
500000/500000 [==============================] - 7s 13us/step - loss: 0.5468 - merge_3_loss: 0.3776 - merge_4_loss: 0.1692 - merge_3_acc: 0.4026 - merge_4_acc: 0.2319 - val_loss: 0.4874 - val_merge_3_loss: 0.2848 - val_merge_4_loss: 0.2026 - val_merge_3_acc: 0.5290 - val_merge_4_acc: 0.2156
Epoch 2/50
500000/500000 [==============================] - 7s 13us/step - loss: 0.4702 - merge_3_loss: 0.2733 - merge_4_loss: 0.1969 - merge_3_acc: 0.5210 - merge_4_acc: 0.2281 - val_loss: 0.4674 - val_merge_3_loss: 0.2696 - val_merge_4_loss: 0.1978 - val_merge_3_acc: 0.5322 - val_merge_4_acc: 0.2126
Epoch 3/50
500000/500000 [==============================] - 7s 14us/step - loss: 0.4540 - merge_3_loss: 0.2604 - merge_4_loss: 0.1936 - merge_3_acc: 0.5402 - merge_4_acc: 0.2280 - val_loss: 0.4551 - val_merge_3_loss: 0.2600 - val_merge_4_loss: 0.1951 - val_merge_3_acc: 0.5462 - val_merge_4_acc: 0.2146
Epoch 4/50
500000/500000 [===============

Epoch 28/50
500000/500000 [==============================] - 6s 12us/step - loss: 0.3682 - merge_3_loss: 0.1973 - merge_4_loss: 0.1709 - merge_3_acc: 0.6914 - merge_4_acc: 0.2284 - val_loss: 0.4157 - val_merge_3_loss: 0.2349 - val_merge_4_loss: 0.1807 - val_merge_3_acc: 0.6316 - val_merge_4_acc: 0.2138
Epoch 29/50
500000/500000 [==============================] - 6s 12us/step - loss: 0.3676 - merge_3_loss: 0.1969 - merge_4_loss: 0.1708 - merge_3_acc: 0.6926 - merge_4_acc: 0.2284 - val_loss: 0.4155 - val_merge_3_loss: 0.2361 - val_merge_4_loss: 0.1794 - val_merge_3_acc: 0.6276 - val_merge_4_acc: 0.2120
Epoch 30/50
500000/500000 [==============================] - 6s 12us/step - loss: 0.3671 - merge_3_loss: 0.1963 - merge_4_loss: 0.1708 - merge_3_acc: 0.6938 - merge_4_acc: 0.2285 - val_loss: 0.4162 - val_merge_3_loss: 0.2373 - val_merge_4_loss: 0.1789 - val_merge_3_acc: 0.6242 - val_merge_4_acc: 0.2108
Epoch 31/50
500000/500000 [==============================] - 6s 11us/step - loss: 0.3666